In [1]:
import pandas as pd
from datetime import date
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import sql, select, join, desc

engine = create_engine('sqlite:///northwind.db')

conn = engine.connect()

DBInfo = MetaData(engine)

DBInfo.tables.keys()

emp = Table('Employees', DBInfo, autoload=True)
order = Table('Orders', DBInfo, autoload=True)
details = Table('Order Details', DBInfo, autoload=True)

#join Orders and Order Details tables into a dataframe and add a new column that shows total cost of order
def orderdf():
    global odf
    orderjoin = select([order.c.OrderID, order.c.EmployeeID, details.c.OrderID, details.c.UnitPrice, details.c.Quantity, details.c.Discount])\
            .select_from(order.join(details))
    orderdetails = conn.execute(orderjoin)
    odf = pd.read_sql(orderjoin, conn)
    odf['Total'] = odf['UnitPrice'] * odf['Quantity'] / (1 - odf['Discount'])

#create dataframe for employee names and hire dates, and a dictionary of full names
def empdf():
    global edf
    global empdict
    emps = select([emp.c.EmployeeID, emp.c.FirstName, emp.c.LastName, emp.c.HireDate])
    edf = pd.read_sql(emps, conn)
    edf['FullName'] = edf['FirstName'] + ' ' + edf['LastName']
    empdict = dict(zip(edf.EmployeeID, edf.FullName))
    

orderdf()
empdf()

#calculate average sales per day for each employee from date of hire to March 29, 2016, which is when the database was uploaded
def avgsales():
    global empID
    global salesdict
    empID = 1
    salesdict = {}
    while empID > 0:
        sales = odf.loc[odf['EmployeeID'] == empID, 'Total'].sum()
        if sales == 0:
            break
        td = date(2016, 3, 29) - edf[edf['EmployeeID'] == empID]['HireDate'].values[0]
        days = td / pd.Timedelta(days=1)
        #print(empID, round(sales / days))
        salesdict[empID] = (round(sales / days))
        empID += 1
    
avgsales()

#combine and sort dictionaries to produce a list of employees from most to least productive
combodict = {}
for i in range(1, empID):
    combodict[salesdict[i]] = empdict[i]
    
sortdict = sorted(combodict.items(), reverse=True)
x = 1
for sales, name in sortdict:
    print(str(x) + '. ' + name + ': $' + str(sales) +' per day' )
    x += 1

1. Margaret Peacock: $256 per day
2. Robert King: $199 per day
3. Laura Callahan: $187 per day
4. Anne Dodsworth: $180 per day
5. Janet Leverling: $155 per day
6. Nancy Davolio: $150 per day
7. Andrew Fuller: $145 per day
8. Steven Buchanan: $94 per day
9. Michael Suyama: $93 per day


/apps/python/3.10/lib/python3.10/site-packages/sqlalchemy/sql/sqltypes.py:661: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(
